In [1]:
import pandas as pd
import numpy as np
from hmmlearn import hmm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from itertools import groupby

In [2]:
# Step 1: Load the dataset
df = pd.read_excel("Data (7).xlsx")

In [3]:
# Encode KeyPressed into numerical values
label_encoder = LabelEncoder()
df['KeyEncoded'] = label_encoder.fit_transform(df['KeyPressed'])

In [4]:
# Select timing features
features = ['FlightTime', 'KeyHoldTime']
df[features] = df[features].fillna(0)  # Handle missing values
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

In [8]:


# Function to train HMM per user and extract features
def train_hmm_per_user(user_df):
    user_df = user_df.copy()  # Ensure a new copy is modified
    X = user_df[['KeyEncoded'] + features].values
    lengths = [len(X)]
    
    hmm_model = hmm.GaussianHMM(n_components=4, covariance_type="diag", n_iter=1000, random_state=42)
    hmm_model.fit(X, lengths)
    
    log_likelihood = hmm_model.score(X)
    hidden_states = hmm_model.predict(X)
    
    user_df['HMM_LogLikelihood_PerUser'] = log_likelihood
    user_df['HMM_HiddenState'] = hidden_states
    
    # Calculate state transition counts
    transitions = np.bincount(hidden_states[:-1] * 4 + hidden_states[1:], minlength=16)
    user_df['HMM_HiddenState_TransitionCount'] = sum(transitions > 0)
    
    # Calculate state duration
    state_durations = [sum(1 for _ in group) for key, group in groupby(hidden_states)]
    user_df['HMM_HiddenState_Duration'] = np.mean(state_durations) if state_durations else 0
    
    return user_df

In [9]:
# Process each group separately and concatenate results
dfs = []
for _, group in df.groupby('DeviceId'):
    dfs.append(train_hmm_per_user(group))

df = pd.concat(dfs).reset_index(drop=True)

Model is not converging.  Current: 29072.22324471682 is not greater than 29072.43045491965. Delta is -0.20721020282871905
Model is not converging.  Current: 16691.809582144895 is not greater than 16693.04972982081. Delta is -1.2401476759150682


In [10]:
# Additional derived features
df['TypingSpeed'] = 1 / (df['FlightTime'].replace(0, np.nan))
df['KeySequenceChangeRate'] = df['HMM_HiddenState_TransitionCount'] / df.groupby('DeviceId')['HMM_HiddenState_TransitionCount'].transform('sum')

df['InterKeyInterval_Mean'] = df.groupby('DeviceId')['FlightTime'].transform('mean')
df['InterKeyInterval_Std'] = df.groupby('DeviceId')['FlightTime'].transform('std')
df['Burstiness'] = df.groupby('DeviceId')['FlightTime'].transform(lambda x: (x.std() - x.mean()) / (x.std() + x.mean()))

df['TransitionEntropy'] = df.groupby('DeviceId')['HMM_HiddenState'].transform(lambda x: -np.sum(np.bincount(x, minlength=4) / len(x) * np.log2(np.bincount(x, minlength=4) / len(x))))
df['DominantHiddenState'] = df.groupby('DeviceId')['HMM_HiddenState'].transform(lambda x: np.bincount(x).argmax())

df['TimeSinceLastKeyPress'] = df.groupby('DeviceId')['FlightTime'].transform(lambda x: x.shift(1).fillna(0).cumsum())
df['AvgSessionLength'] = df.groupby('DeviceId')['FlightTime'].transform(lambda x: (x > 2 * x.mean()).cumsum().value_counts().mean())


In [11]:
# Save enhanced dataset
df.to_csv("enhanced_dataset.csv", index=False)
print("Feature extraction complete. Saved to enhanced_dataset.csv")

Feature extraction complete. Saved to enhanced_dataset.csv


# Data Dictionary

## Original Columns

| Column Name      | Description |
|-----------------|-------------|
| **DeviceId** | Unique identifier for each user/device. |
| **KeyPressed** | The key that was pressed by the user. |
| **FlightTime** | Time taken between releasing one key and pressing the next (milliseconds). |
| **KeyHoldTime** | Duration for which a key is held down before release (milliseconds). |

## Derived Columns

| Column Name | Description |
|-------------|-------------|
| **KeyEncoded** | Encoded numerical representation of `KeyPressed` using Label Encoding. |
| **HMM_LogLikelihood_PerUser** | Log-likelihood of the trained HMM model per user, representing how well the model fits the user’s typing pattern. |
| **HMM_HiddenState** | Hidden state assigned to each keystroke based on the trained HMM model. |
| **HMM_HiddenState_TransitionCount** | Number of transitions between hidden states, indicating typing variability. |
| **HMM_HiddenState_Duration** | Average duration (number of keystrokes) spent in a given hidden state before transitioning. |
| **TypingSpeed** | Estimated typing speed, computed as `1 / FlightTime`. (Higher values indicate faster typing.) |
| **KeySequenceChangeRate** | Ratio of hidden state transitions to the total transitions per user, capturing consistency in typing patterns. |
| **InterKeyInterval_Mean** | Mean time interval between consecutive keystrokes per user. |
| **InterKeyInterval_Std** | Standard deviation of inter-key intervals, indicating typing rhythm variability. |
| **Burstiness** | A measure of irregularity in typing speed, computed using `(std - mean) / (std + mean)`. |
| **TransitionEntropy** | Shannon entropy of state transitions, measuring typing pattern complexity. |
| **DominantHiddenState** | Most frequently occurring hidden state for each user, indicating the most common typing pattern. |
| **TimeSinceLastKeyPress** | Cumulative time elapsed since the last key press per user. |
| **AvgSessionLength** | Estimated session length based on typing pauses (breaks longer than twice the mean `FlightTime` are considered session boundaries). |

